# Fine-tune own financial sentiment NLP

In [1]:
import numpy as np
import pandas as pd 
from transformers import BertTokenizer, Trainer, BertForSequenceClassification, TrainingArguments
from datasets import Dataset
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# tested in transformers==4.18.0, pytorch==1.7.1 
import torch
import transformers
torch.__version__, transformers.__version__

('2.3.0+cu121', '4.41.1')

In [3]:
torch.cuda.is_available()

True

# Data loading

 '1' denotes a positive sentiment, '2' signifies a negative sentiment, and '0' indicates a neutral sentiment. 

In [92]:
df1 = pd.read_parquet('fingpt_sentiment_train-00000-of-00001-7790814d50128a07.parquet')

df1

,text,label
0,"Teollisuuden Voima Oyj , the Finnish utility k...",neutral
1,Sanofi poaches AstraZeneca scientist as new re...,neutral
2,Starbucks says the workers violated safety pol...,negative
3,$brcm raises revenue forecast,positive
4,Google parent Alphabet Inc. reported revenue a...,negative
...,...,...
76767,"BP, Statoil, to Withdraw Staff From Algeria Fo...",negative
76768,NEW YORK — A fire broke out late Wednesday at ...,negative
76769,Operating profit margin increased from 11.2 % ...,positive
76770,$vxx adding to position here !,positive


In [93]:
label_mapping = {'neutral': 0, 'positive': 1, 'negative': 2}

# Apply the mapping to the 'label' column
df1['label'] = df1['label'].map(label_mapping)

df1

,text,label
0,"Teollisuuden Voima Oyj , the Finnish utility k...",0
1,Sanofi poaches AstraZeneca scientist as new re...,0
2,Starbucks says the workers violated safety pol...,2
3,$brcm raises revenue forecast,1
4,Google parent Alphabet Inc. reported revenue a...,2
...,...,...
76767,"BP, Statoil, to Withdraw Staff From Algeria Fo...",2
76768,NEW YORK — A fire broke out late Wednesday at ...,2
76769,Operating profit margin increased from 11.2 % ...,1
76770,$vxx adding to position here !,1


In [94]:
df2 = pd.read_parquet('financial_phrasebank-00000-of-00001-46d5c1a7817abe3d.parquet')

# Apply the mapping to the 'label' column
df2['label'] = df2['label'].map(label_mapping)

df2

,text,label
0,"According to Gran , the company has no plans t...",0
1,Technopolis plans to develop in stages an area...,0
2,With the new production plant the company woul...,1
3,According to the company 's updated strategy f...,1
4,"For the last quarter of 2010 , Componenta 's n...",1
...,...,...
4212,HELSINKI Thomson Financial - Shares in Cargote...,2
4213,LONDON MarketWatch -- Share prices ended lower...,2
4214,Rinkuskiai 's beer sales fell by 6.5 per cent ...,0
4215,Operating profit fell to EUR 35.4 mn from EUR ...,2


In [95]:
df3= pd.read_parquet('auditor_sentiment-00000-of-00001-7c7834e29b677695.parquet')

# Apply the mapping to the 'label' column
df3['label'] = df3['label'].map(label_mapping)

df3

,text,label
0,Altia 's operating profit jumped to EUR 47 mil...,1
1,The agreement was signed with Biohit Healthcar...,1
2,"Kesko pursues a strategy of healthy , focused ...",1
3,"Vaisala , headquartered in Helsinki in Finland...",0
4,"Also , a six-year historic analysis is provide...",0
...,...,...
3872,Dubai Nokia has announced the launch of `` Com...,1
3873,"MADISON , Wis. , Feb. 6 - PRNewswire - -- Fisk...",1
3874,The report provides a comprehensive insight in...,1
3875,Pharmaceuticals - Netherlands This brand-new m...,0


In [96]:
df4 = pd.read_parquet('twitter_financial_news_sentiment-00000-of-00001-e4ff9b93e3f0bcb7.parquet')

label_mapping = {'neutral': 0, 'positive': 1, 'negative': 2}

# Apply the mapping to the 'label' column
df4['label'] = df4['label'].map(label_mapping)

df4

,text,label
0,$BYND - JPMorgan reels in expectations on Beyo...,2
1,$CCL $RCL - Nomura points to bookings weakness...,2
2,"$CX - Cemex cut at Credit Suisse, J.P. Morgan ...",2
3,$ESS: BTIG Research cuts to Neutral https://t....,2
4,$FNKO - Funko slides after Piper Jaffray PT cu...,2
...,...,...
9538,The Week's Gainers and Losers on the Stoxx Eur...,0
9539,Tupperware Brands among consumer gainers; Unil...,0
9540,vTv Therapeutics leads healthcare gainers; Myo...,0
9541,"WORK, XPO, PYX and AMKR among after hour movers",0


In [97]:
# Try reading the file with a different encoding
df5 = pd.read_csv('sentiment_finance.csv', encoding='latin1')

# Apply the label mapping
label_mapping = {'neutral': 0, 'positive': 1, 'negative': 2}
df5['label'] = df5['label'].map(label_mapping)
df5

,text,label
0,"According to Gran , the company has no plans t...",0
1,Technopolis plans to develop in stages an area...,0
2,The international electronic industry company ...,2
3,With the new production plant the company woul...,1
4,According to the company 's updated strategy f...,1
...,...,...
4841,LONDON MarketWatch -- Share prices ended lower...,2
4842,Rinkuskiai 's beer sales fell by 6.5 per cent ...,0
4843,Operating profit fell to EUR 35.4 mn from EUR ...,2
4844,Net sales of the Paper segment decreased to EU...,2


In [98]:
df6 = pd.read_csv('Financial_News_Sentiment.csv', encoding='latin1')
df6 = df6.rename(columns={'text': 'text', 'sentiment': 'label'})
df6['label'] = df6['label'].replace({0: 2, 1: 0, 2: 1})
df6

,text,label
0,Global COVID-19 death toll exceeds 4 million.,2
1,"reports 67,208 new COVID-19 cases, 2,330 deaths.",2
2,China reports 23 new COVID-19 cases versus 19 ...,2
3,"India records 91,702 new COVID-19 cases over p...",2
4,sharply raises COVID-19 death toll prompting c...,2
...,...,...
206,U.S. equity futures were trading higher the mo...,1
207,"China, US commerce chiefs to cooperate on hand...",1
208,Stock investors celebrate red-hot five-quarter...,1
209,Wall Streetâs roaring 2021 is as good as it ...,1


In [99]:
df7 = pd.read_csv('data_2.csv')
df7.dropna(how='all')
df7 = df7.rename(columns={'Sentence': 'text', 'Sentiment': 'label'})
# Apply the label mapping
label_mapping = {'neutral': 0, 'positive': 1, 'negative': 2}
df7['label'] = df7['label'].map(label_mapping)
df7

,text,label
0,The GeoSolutions technology will leverage Bene...,1
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",2
2,"For the last quarter of 2010 , Componenta 's n...",1
3,According to the Finnish-Russian Chamber of Co...,0
4,The Swedish buyout firm has sold its remaining...,0
...,...,...
5837,RISING costs have forced packaging producer Hu...,2
5838,Nordic Walking was first used as a summer trai...,0
5839,"According shipping company Viking Line , the E...",0
5840,"In the building and home improvement trade , s...",0


In [100]:
df8 = pd.read_parquet('fiqa_2018-00000-of-00001-ea3e4fffb9862c66.parquet')
# Apply the label mapping
label_mapping = {'neutral': 0, 'positive': 1, 'negative': 2}
df8['label'] = df8['label'].map(label_mapping)
df8

,text,label
0,Still short $LNG from $11.70 area...next stop ...,2
1,$PLUG bear raid,2
2,How Kraft-Heinz Merger Came Together in Speedy...,1
3,Slump in Weir leads FTSE down from record high,2
4,"$AAPL bounces off support, it seems",1
...,...,...
956,Morrisons finance chief to fill gap as CEO lea...,2
957,End Of Day Scan: Bullish MA Crossovers $BBVA ...,1
958,Japan's Nikkei lands Financial Times in $1.3 b...,1
959,Double bottom with handle buy point of 56.49 $...,1


In [103]:
combined_df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], ignore_index=True)
combined_df.describe()

,label
count,106269.000000
mean,0.764475
std,0.762507
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,2.000000


# Sequential Training

In [ ]:
# train df1 and df2 now
dataframes = [df1, df2]

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-pretrain')
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-pretrain', num_labels=3)

# Define compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': accuracy_score(predictions, labels)}

# Define training arguments
args = TrainingArguments(
    output_dir='temp/',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=1,  # Train for 1 epoch per dataframe
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
)

# Training loop over individual dataframes
for idx, df in enumerate(dataframes):
    print(f"Training on dataframe {idx+1}/{len(dataframes)}")
    
    # Split dataframe into train, validation, and test sets
    df_train, df_test = train_test_split(df, stratify=df['label'], test_size=0.2, random_state=42)
    df_train, df_val = train_test_split(df_train, stratify=df_train['label'], test_size=0.2, random_state=42)
    
    # Convert to datasets
    dataset_train = Dataset.from_pandas(df_train)
    dataset_val = Dataset.from_pandas(df_val)
    dataset_test = Dataset.from_pandas(df_test)
    
    # Tokenize datasets
    dataset_train = dataset_train.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
    dataset_val = dataset_val.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
    dataset_test = dataset_test.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
    
    # Set format for PyTorch
    dataset_train.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
    dataset_val.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
    dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
    
    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=dataset_train,
        eval_dataset=dataset_val,
        compute_metrics=compute_metrics
    )
    
    # Train the model
    trainer.train()
    
    # Save the model after each dataframe
    model.save_pretrained(f'temp/model_df_{idx+1}')
    tokenizer.save_pretrained(f'temp/model_df_{idx+1}')
    
    # Optionally, evaluate on the test set after each dataframe
    eval_results = trainer.evaluate(eval_dataset=dataset_test)
    print(f"Eval results for dataframe {idx+1}: {eval_results}")

# try using model 2


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

# Load the saved model and tokenizer
model = BertForSequenceClassification.from_pretrained('temp/model_df_2')
tokenizer = BertTokenizer.from_pretrained('temp/model_df_2')


In [ ]:
import torch

# Sample texts for prediction
texts = ["The company reported a significant increase in revenue.", 
         "The stock price dropped after the announcement of the new policy."]

# Tokenize the input texts
inputs = tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors='pt')

# Make predictions
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    outputs = model(**inputs)

# Get the predicted class labels
predictions = torch.argmax(outputs.logits, dim=1)

# Convert predictions to human-readable labels (assuming 0, 1, 2 correspond to the sentiment classes)
label_map = {0: 'neutral', 1: 'positive', 2: 'negative'}
predicted_labels = [label_map[pred.item()] for pred in predictions]

# Print the predictions
for text, label in zip(texts, predicted_labels):
    print(f"Text: {text}\nPredicted label: {label}\n")


In [ ]:
df_testing = pd.read_parquet('fiqa_2018-00000-of-00001-ea3e4fffb9862c66.parquet')
label_mapping = {'neutral': 0, 'positive': 1, 'negative': 2}

# Apply the mapping to the 'label' column
df_testing['label'] = df_testing['label'].map(label_mapping)

df_testing

In [ ]:
from transformers import Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd

# Convert to Hugging Face Dataset
dataset_test = Dataset.from_pandas(df_testing)

# Tokenize the test dataset
dataset_test = dataset_test.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

# Define compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': accuracy_score(predictions, labels)}

# Define training arguments for evaluation
args = TrainingArguments(
    output_dir='temp/',
    per_device_eval_batch_size=32,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=args,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics
)

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Eval results: {eval_results}")


In [5]:
df_testing2 = pd.read_csv('test.csv')
df_testing2 = df_testing2.rename(columns={'Sentence': 'text', 'Sentiment': 'label'})
# Replace the values in the 'label' column
df_testing2['label'] = df_testing2['label'].replace({0: 2, 1: 0, 2: 1})
df_testing2

,text,label
0,A few employees would remain at the Oulu plant...,0
1,Comparable net sales are expected to increase ...,1
2,"Tesla is recalling 2,700 Model X cars: https:/...",2
3,Finnish software developer Done Solutions Oyj ...,1
4,Compagnie de Financement Foncier - Is to issue...,0
...,...,...
1164,"In 2010 , the Marimekko Group s net sales were...",0
1165,Nokia said it still expects to sell 150 more m...,0
1166,"PNC, Goldman Sachs Receive Dividend Hike Green...",1
1167,"The company , employing 6,400 , reported net s...",0


In [ ]:
# Convert to Hugging Face Dataset
dataset_test = Dataset.from_pandas(df_testing2)

# Tokenize the test dataset
dataset_test = dataset_test.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

# Define compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': accuracy_score(predictions, labels)}

# Define training arguments for evaluation
args = TrainingArguments(
    output_dir='temp/',
    per_device_eval_batch_size=32,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=args,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics
)

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Eval results: {eval_results}")

# keep training df3

In [ ]:
dataframes = [df3, df4]

# Load the last saved model
model = BertForSequenceClassification.from_pretrained('temp/model_df_2')
tokenizer = BertTokenizer.from_pretrained('temp/model_df_2')

# Continue training loop over the next set of dataframes
for idx, df in enumerate(dataframes, start=2):  # Start index from 2 to continue numbering
    print(f"Training on dataframe {idx+1}/{len(dataframes)+2}")
    
    # Split dataframe into train, validation, and test sets
    df_train, df_test = train_test_split(df, stratify=df['label'], test_size=0.2, random_state=42)
    df_train, df_val = train_test_split(df_train, stratify=df_train['label'], test_size=0.2, random_state=42)
    
    # Convert to datasets
    dataset_train = Dataset.from_pandas(df_train)
    dataset_val = Dataset.from_pandas(df_val)
    dataset_test = Dataset.from_pandas(df_test)
    
    # Tokenize datasets
    dataset_train = dataset_train.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
    dataset_val = dataset_val.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
    dataset_test = dataset_test.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
    
    # Set format for PyTorch
    dataset_train.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
    dataset_val.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
    dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
    
    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=dataset_train,
        eval_dataset=dataset_val,
        compute_metrics=compute_metrics
    )
    
    # Train the model
    trainer.train()
    
    # Save the model after each dataframe
    model.save_pretrained(f'temp/model_df_{idx+1}')
    tokenizer.save_pretrained(f'temp/model_df_{idx+1}')
    
    # Optionally, evaluate on the test set after each dataframe
    eval_results = trainer.evaluate(eval_dataset=dataset_test)
    print(f"Eval results for dataframe {idx+1}: {eval_results}")

# try using model 4

In [ ]:
# Load the saved model and tokenizer
model = BertForSequenceClassification.from_pretrained('temp/model_df_4')
tokenizer = BertTokenizer.from_pretrained('temp/model_df_4')

In [ ]:
# Convert to Hugging Face Dataset
dataset_test = Dataset.from_pandas(df_testing)

# Tokenize the test dataset
dataset_test = dataset_test.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

# Define compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': accuracy_score(predictions, labels)}

# Define training arguments for evaluation
args = TrainingArguments(
    output_dir='temp/',
    per_device_eval_batch_size=32,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=args,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics
)

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Eval results: {eval_results}")


In [ ]:
# Convert to Hugging Face Dataset
dataset_test = Dataset.from_pandas(df_testing2)

# Tokenize the test dataset
dataset_test = dataset_test.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

# Define compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': accuracy_score(predictions, labels)}

# Define training arguments for evaluation
args = TrainingArguments(
    output_dir='temp/',
    per_device_eval_batch_size=32,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=args,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics
)

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Eval results: {eval_results}")

# keep training df5

In [ ]:
dataframes = [df5]

# Load the last saved model
model = BertForSequenceClassification.from_pretrained('temp/model_df_4')
tokenizer = BertTokenizer.from_pretrained('temp/model_df_4')

# Continue training loop over the next set of dataframes
for idx, df in enumerate(dataframes, start=2):  # Start index from 2 to continue numbering
    print(f"Training on dataframe {idx+1}/{len(dataframes)+2}")
    
    # Split dataframe into train, validation, and test sets
    df_train, df_test = train_test_split(df, stratify=df['label'], test_size=0.2, random_state=42)
    df_train, df_val = train_test_split(df_train, stratify=df_train['label'], test_size=0.2, random_state=42)
    
    # Convert to datasets
    dataset_train = Dataset.from_pandas(df_train)
    dataset_val = Dataset.from_pandas(df_val)
    dataset_test = Dataset.from_pandas(df_test)
    
    # Tokenize datasets
    dataset_train = dataset_train.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
    dataset_val = dataset_val.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
    dataset_test = dataset_test.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
    
    # Set format for PyTorch
    dataset_train.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
    dataset_val.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
    dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
    
    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=dataset_train,
        eval_dataset=dataset_val,
        compute_metrics=compute_metrics
    )
    
    # Train the model
    trainer.train()
    
    # Save the model after each dataframe
    model.save_pretrained(f'temp/model_df_{idx+1}')
    tokenizer.save_pretrained(f'temp/model_df_{idx+1}')
    
    # Optionally, evaluate on the test set after each dataframe
    eval_results = trainer.evaluate(eval_dataset=dataset_test)
    print(f"Eval results for dataframe {idx+1}: {eval_results}")

In [ ]:
model.save_pretrained(f'temp/model_df_5')
tokenizer.save_pretrained(f'temp/model_df_5')

# try using model 5

In [ ]:
# Load the saved model and tokenizer
model = BertForSequenceClassification.from_pretrained('temp/model_df_5')
tokenizer = BertTokenizer.from_pretrained('temp/model_df_5')

In [ ]:
# Convert to Hugging Face Dataset
dataset_test = Dataset.from_pandas(df_testing)

# Tokenize the test dataset
dataset_test = dataset_test.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

# Define compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': accuracy_score(predictions, labels)}

# Define training arguments for evaluation
args = TrainingArguments(
    output_dir='temp/',
    per_device_eval_batch_size=32,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=args,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics
)

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Eval results: {eval_results}")


In [ ]:
# Convert to Hugging Face Dataset
dataset_test = Dataset.from_pandas(df_testing2)

# Tokenize the test dataset
dataset_test = dataset_test.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

# Define compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': accuracy_score(predictions, labels)}

# Define training arguments for evaluation
args = TrainingArguments(
    output_dir='temp/',
    per_device_eval_batch_size=32,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=args,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics
)

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Eval results: {eval_results}")

# keep training df6

In [ ]:
dataframes = [df6]

# Load the last saved model
model = BertForSequenceClassification.from_pretrained('temp/model_df_5')
tokenizer = BertTokenizer.from_pretrained('temp/model_df_5')

# Continue training loop over the next set of dataframes
for idx, df in enumerate(dataframes, start=2):  # Start index from 2 to continue numbering
    print(f"Training on dataframe {idx+1}/{len(dataframes)+2}")
    
    # Split dataframe into train, validation, and test sets
    df_train, df_test = train_test_split(df, stratify=df['label'], test_size=0.2, random_state=42)
    df_train, df_val = train_test_split(df_train, stratify=df_train['label'], test_size=0.2, random_state=42)
    
    # Convert to datasets
    dataset_train = Dataset.from_pandas(df_train)
    dataset_val = Dataset.from_pandas(df_val)
    dataset_test = Dataset.from_pandas(df_test)
    
    # Tokenize datasets
    dataset_train = dataset_train.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
    dataset_val = dataset_val.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
    dataset_test = dataset_test.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
    
    # Set format for PyTorch
    dataset_train.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
    dataset_val.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
    dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
    
    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=dataset_train,
        eval_dataset=dataset_val,
        compute_metrics=compute_metrics
    )
    
    # Train the model
    trainer.train()
    
    # Save the model after each dataframe
    model.save_pretrained(f'temp/model_df_{idx+1}')
    tokenizer.save_pretrained(f'temp/model_df_{idx+1}')
    
    # Optionally, evaluate on the test set after each dataframe
    eval_results = trainer.evaluate(eval_dataset=dataset_test)
    print(f"Eval results for dataframe {idx+1}: {eval_results}")

In [ ]:
model.save_pretrained(f'temp/model_df_6')
tokenizer.save_pretrained(f'temp/model_df_6')

# try using model 6

In [ ]:
# Load the saved model and tokenizer
model = BertForSequenceClassification.from_pretrained('temp/model_df_6')
tokenizer = BertTokenizer.from_pretrained('temp/model_df_6')

In [ ]:
# Convert to Hugging Face Dataset
dataset_test = Dataset.from_pandas(df_testing)

# Tokenize the test dataset
dataset_test = dataset_test.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

# Define compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': accuracy_score(predictions, labels)}

# Define training arguments for evaluation
args = TrainingArguments(
    output_dir='temp/',
    per_device_eval_batch_size=32,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=args,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics
)

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Eval results: {eval_results}")

In [ ]:
# Convert to Hugging Face Dataset
dataset_test = Dataset.from_pandas(df_testing2)

# Tokenize the test dataset
dataset_test = dataset_test.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

# Define compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': accuracy_score(predictions, labels)}

# Define training arguments for evaluation
args = TrainingArguments(
    output_dir='temp/',
    per_device_eval_batch_size=32,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=args,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics
)

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Eval results: {eval_results}")

# keep training df7

In [52]:
dataframes = [df7]

# Load the last saved model
model = BertForSequenceClassification.from_pretrained('temp/model_df_6')
tokenizer = BertTokenizer.from_pretrained('temp/model_df_6')

# Continue training loop over the next set of dataframes
for idx, df in enumerate(dataframes, start=2):  # Start index from 2 to continue numbering
    print(f"Training on dataframe {idx+1}/{len(dataframes)+2}")
    
    # Split dataframe into train, validation, and test sets
    df_train, df_test = train_test_split(df, stratify=df['label'], test_size=0.2, random_state=42)
    df_train, df_val = train_test_split(df_train, stratify=df_train['label'], test_size=0.2, random_state=42)
    
    # Convert to datasets
    dataset_train = Dataset.from_pandas(df_train)
    dataset_val = Dataset.from_pandas(df_val)
    dataset_test = Dataset.from_pandas(df_test)
    
    # Tokenize datasets
    dataset_train = dataset_train.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
    dataset_val = dataset_val.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
    dataset_test = dataset_test.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
    
    # Set format for PyTorch
    dataset_train.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
    dataset_val.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
    dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
    
    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=dataset_train,
        eval_dataset=dataset_val,
        compute_metrics=compute_metrics
    )
    
    # Train the model
    trainer.train()
    
    # Save the model after each dataframe
    model.save_pretrained(f'temp/model_df_{idx+1}')
    tokenizer.save_pretrained(f'temp/model_df_{idx+1}')
    
    # Optionally, evaluate on the test set after each dataframe
    eval_results = trainer.evaluate(eval_dataset=dataset_test)
    print(f"Eval results for dataframe {idx+1}: {eval_results}")

Training on dataframe 3/3


Map:   0%|          | 0/3738 [00:00<?, ? examples/s]

Map:   0%|          | 0/935 [00:00<?, ? examples/s]

Map:   0%|          | 0/1169 [00:00<?, ? examples/s]

Step,Training Loss
500,0.377300
1000,0.207800


Eval results for dataframe 3: {'eval_loss': 0.37617334723472595, 'eval_accuracy': 0.8665526090675791, 'eval_runtime': 7.8681, 'eval_samples_per_second': 148.575, 'eval_steps_per_second': 4.703, 'epoch': 3.0}


In [53]:
model.save_pretrained(f'temp/model_df_7')
tokenizer.save_pretrained(f'temp/model_df_7')

('temp/model_df_7/tokenizer_config.json',
 'temp/model_df_7/special_tokens_map.json',
 'temp/model_df_7/vocab.txt',
 'temp/model_df_7/added_tokens.json')

In [54]:
# Load the saved model and tokenizer
model = BertForSequenceClassification.from_pretrained('temp/model_df_7')
tokenizer = BertTokenizer.from_pretrained('temp/model_df_7')

In [55]:
# Convert to Hugging Face Dataset
dataset_test = Dataset.from_pandas(df_testing2)

# Tokenize the test dataset
dataset_test = dataset_test.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

# Define compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': accuracy_score(predictions, labels)}

# Define training arguments for evaluation
args = TrainingArguments(
    output_dir='temp/',
    per_device_eval_batch_size=32,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=args,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics
)

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Eval results: {eval_results}")

Map:   0%|          | 0/1169 [00:00<?, ? examples/s]

Eval results: {'eval_loss': 0.17151209712028503, 'eval_accuracy': 0.9084687767322498, 'eval_runtime': 7.7511, 'eval_samples_per_second': 150.816, 'eval_steps_per_second': 4.773}


# keep training df8

In [85]:
dataframes = [df8]

# Load the last saved model
model = BertForSequenceClassification.from_pretrained('temp/model_df_7')
tokenizer = BertTokenizer.from_pretrained('temp/model_df_7')

# Continue training loop over the next set of dataframes
for idx, df in enumerate(dataframes, start=2):  # Start index from 2 to continue numbering
    print(f"Training on dataframe {idx+1}/{len(dataframes)+2}")
    
    # Split dataframe into train, validation, and test sets
    df_train, df_test = train_test_split(df, stratify=df['label'], test_size=0.2, random_state=42)
    df_train, df_val = train_test_split(df_train, stratify=df_train['label'], test_size=0.2, random_state=42)
    
    # Convert to datasets
    dataset_train = Dataset.from_pandas(df_train)
    dataset_val = Dataset.from_pandas(df_val)
    dataset_test = Dataset.from_pandas(df_test)
    
    # Tokenize datasets
    dataset_train = dataset_train.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
    dataset_val = dataset_val.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
    dataset_test = dataset_test.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
    
    # Set format for PyTorch
    dataset_train.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
    dataset_val.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
    dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
    
    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=dataset_train,
        eval_dataset=dataset_val,
        compute_metrics=compute_metrics
    )
    
    # Train the model
    trainer.train()
    
    # Save the model after each dataframe
    model.save_pretrained(f'temp/model_df_{idx+1}')
    tokenizer.save_pretrained(f'temp/model_df_{idx+1}')
    
    # Optionally, evaluate on the test set after each dataframe
    eval_results = trainer.evaluate(eval_dataset=dataset_test)
    print(f"Eval results for dataframe {idx+1}: {eval_results}")

Training on dataframe 3/3


Map:   0%|          | 0/614 [00:00<?, ? examples/s]

Map:   0%|          | 0/154 [00:00<?, ? examples/s]

Map:   0%|          | 0/193 [00:00<?, ? examples/s]

Step,Training Loss


Eval results for dataframe 3: {'eval_loss': 0.27275797724723816, 'eval_accuracy': 0.9481865284974094, 'eval_runtime': 1.3352, 'eval_samples_per_second': 144.55, 'eval_steps_per_second': 5.243, 'epoch': 3.0}


In [86]:
model.save_pretrained(f'temp/model_df_8')
tokenizer.save_pretrained(f'temp/model_df_8')

('temp/model_df_8/tokenizer_config.json',
 'temp/model_df_8/special_tokens_map.json',
 'temp/model_df_8/vocab.txt',
 'temp/model_df_8/added_tokens.json')

In [87]:
# Load the saved model and tokenizer
model = BertForSequenceClassification.from_pretrained('temp/model_df_8')
tokenizer = BertTokenizer.from_pretrained('temp/model_df_8')

In [88]:
# Convert to Hugging Face Dataset
dataset_test = Dataset.from_pandas(df_testing2)

# Tokenize the test dataset
dataset_test = dataset_test.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

# Define compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': accuracy_score(predictions, labels)}

# Define training arguments for evaluation
args = TrainingArguments(
    output_dir='temp/',
    per_device_eval_batch_size=32,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=args,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics
)

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Eval results: {eval_results}")

Map:   0%|          | 0/1169 [00:00<?, ? examples/s]

Eval results: {'eval_loss': 0.3001035153865814, 'eval_accuracy': 0.8964927288280582, 'eval_runtime': 7.567, 'eval_samples_per_second': 154.486, 'eval_steps_per_second': 4.89}


# Evaluate overall models

In [90]:
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
from datasets import Dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd

# Load the test dataset
dataset_test = Dataset.from_pandas(df_testing2)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Define training arguments for evaluation
args = TrainingArguments(
    output_dir='temp/',
    per_device_eval_batch_size=32,
)

# List of model checkpoints
model_checkpoints = [f'temp/model_df_{i}' for i in range(1, 9)]

# Tokenize the test dataset once
tokenizer = BertTokenizer.from_pretrained(model_checkpoints[0])
tokenized_dataset_test = dataset_test.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
tokenized_dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

# Dictionary to store evaluation results
eval_results_dict = {}

for checkpoint in model_checkpoints:
    # Load the saved model
    model = BertForSequenceClassification.from_pretrained(checkpoint)
    
    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=args,
        eval_dataset=tokenized_dataset_test,
        compute_metrics=compute_metrics
    )
    
    # Evaluate the model
    eval_results = trainer.evaluate()
    eval_results_dict[checkpoint] = eval_results
    print(f"Eval results for {checkpoint}: {eval_results}")

eval_results_df = pd.DataFrame(eval_results_dict).T
print(eval_results_df)

Map:   0%|          | 0/1169 [00:00<?, ? examples/s]

Eval results for temp/model_df_1: {'eval_loss': 0.4753454625606537, 'eval_accuracy': 0.8887938408896493, 'eval_precision': 0.91685506028011, 'eval_recall': 0.8887938408896493, 'eval_f1': 0.8945557132392937, 'eval_runtime': 8.018, 'eval_samples_per_second': 145.798, 'eval_steps_per_second': 4.615}


Eval results for temp/model_df_2: {'eval_loss': 0.4671970307826996, 'eval_accuracy': 0.8922155688622755, 'eval_precision': 0.9160435261539863, 'eval_recall': 0.8922155688622755, 'eval_f1': 0.8974479692377935, 'eval_runtime': 8.3101, 'eval_samples_per_second': 140.672, 'eval_steps_per_second': 4.452}


Eval results for temp/model_df_3: {'eval_loss': 0.3001035153865814, 'eval_accuracy': 0.8964927288280582, 'eval_precision': 0.9251385140263982, 'eval_recall': 0.8964927288280582, 'eval_f1': 0.9009000197422004, 'eval_runtime': 8.4479, 'eval_samples_per_second': 138.378, 'eval_steps_per_second': 4.38}


Eval results for temp/model_df_4: {'eval_loss': 1.4409071207046509, 'eval_accuracy': 0.7750213857998289, 'eval_precision': 0.8011277843465486, 'eval_recall': 0.7750213857998289, 'eval_f1': 0.7769965119923422, 'eval_runtime': 8.2165, 'eval_samples_per_second': 142.275, 'eval_steps_per_second': 4.503}


Eval results for temp/model_df_5: {'eval_loss': 1.1273574829101562, 'eval_accuracy': 0.853721129170231, 'eval_precision': 0.8788014041491926, 'eval_recall': 0.853721129170231, 'eval_f1': 0.8602961162053503, 'eval_runtime': 7.9587, 'eval_samples_per_second': 146.883, 'eval_steps_per_second': 4.649}


Eval results for temp/model_df_6: {'eval_loss': 0.8956769704818726, 'eval_accuracy': 0.8383233532934131, 'eval_precision': 0.8704024996943253, 'eval_recall': 0.8383233532934131, 'eval_f1': 0.8463496710883314, 'eval_runtime': 7.7943, 'eval_samples_per_second': 149.981, 'eval_steps_per_second': 4.747}


Eval results for temp/model_df_7: {'eval_loss': 0.17151209712028503, 'eval_accuracy': 0.9084687767322498, 'eval_precision': 0.9277880217020845, 'eval_recall': 0.9084687767322498, 'eval_f1': 0.9132672342615976, 'eval_runtime': 7.6839, 'eval_samples_per_second': 152.137, 'eval_steps_per_second': 4.815}


Eval results for temp/model_df_8: {'eval_loss': 0.3001035153865814, 'eval_accuracy': 0.8964927288280582, 'eval_precision': 0.9251385140263982, 'eval_recall': 0.8964927288280582, 'eval_f1': 0.9009000197422004, 'eval_runtime': 7.6205, 'eval_samples_per_second': 153.401, 'eval_steps_per_second': 4.855}
                 eval_loss  eval_accuracy  eval_precision  eval_recall  \
temp/model_df_1   0.475345       0.888794        0.916855     0.888794   
temp/model_df_2   0.467197       0.892216        0.916044     0.892216   
temp/model_df_3   0.300104       0.896493        0.925139     0.896493   
temp/model_df_4   1.440907       0.775021        0.801128     0.775021   
temp/model_df_5   1.127357       0.853721        0.878801     0.853721   
temp/model_df_6   0.895677       0.838323        0.870402     0.838323   
temp/model_df_7   0.171512       0.908469        0.927788     0.908469   
temp/model_df_8   0.300104       0.896493        0.925139     0.896493   

                  eval_f1  eval_

# Compare before and after fine tune

In [107]:
# Load the fine-tuned model
fine_tuned_model = BertForSequenceClassification.from_pretrained('temp/model_df_7')

# Load the pre-trained model
pretrained_model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-pretrain', num_labels=3)

# Access classifier weights and biases
fine_tuned_classifier_weights = fine_tuned_model.classifier.weight
fine_tuned_classifier_bias = fine_tuned_model.classifier.bias

pretrained_classifier_weights = pretrained_model.classifier.weight
pretrained_classifier_bias = pretrained_model.classifier.bias

print("Fine-tuned Classifier Weights:")
print(fine_tuned_classifier_weights)

print("Pre-trained Classifier Weights:")
print(pretrained_classifier_weights)

print("Fine-tuned Classifier Bias:")
print(fine_tuned_classifier_bias)

print("Pre-trained Classifier Bias:")
print(pretrained_classifier_bias)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at yiyanghkust/finbert-pretrain and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Fine-tuned Classifier Weights:
Parameter containing:
tensor([[-0.0198, -0.0209,  0.0135,  ...,  0.0032, -0.0268, -0.0092],
        [-0.0038, -0.0017,  0.0402,  ...,  0.0201,  0.0100,  0.0358],
        [-0.0035,  0.0289,  0.0178,  ...,  0.0109,  0.0106, -0.0021]],
       requires_grad=True)
Pre-trained Classifier Weights:
Parameter containing:
tensor([[-0.0099,  0.0073,  0.0068,  ..., -0.0317,  0.0128, -0.0054],
        [-0.0122,  0.0022,  0.0025,  ...,  0.0041,  0.0189, -0.0233],
        [ 0.0128,  0.0216, -0.0242,  ...,  0.0176, -0.0040,  0.0109]],
       requires_grad=True)
Fine-tuned Classifier Bias:
Parameter containing:
tensor([ 0.0011, -0.0011, -0.0017], requires_grad=True)
Pre-trained Classifier Bias:
Parameter containing:
tensor([0., 0., 0.], requires_grad=True)


In [108]:
fine_tuned_encoder_layer = fine_tuned_model.bert.encoder.layer[0]
pretrained_encoder_layer = pretrained_model.bert.encoder.layer[0]

fine_tuned_query_weights = fine_tuned_encoder_layer.attention.self.query.weight
pretrained_query_weights = pretrained_encoder_layer.attention.self.query.weight

print("Fine-tuned Encoder Query Weights (Layer 0):")
print(fine_tuned_query_weights)

print("Pre-trained Encoder Query Weights (Layer 0):")
print(pretrained_query_weights)

fine_tuned_key_weights = fine_tuned_encoder_layer.attention.self.key.weight
pretrained_key_weights = pretrained_encoder_layer.attention.self.key.weight

print("Fine-tuned Encoder Key Weights (Layer 0):")
print(fine_tuned_key_weights)

print("Pre-trained Encoder Key Weights (Layer 0):")
print(pretrained_key_weights)

Fine-tuned Encoder Query Weights (Layer 0):
Parameter containing:
tensor([[-5.6009e-02,  3.2049e-02, -5.5241e-03,  ...,  2.2374e-02,
          1.1440e-02,  5.3082e-03],
        [ 3.0205e-02,  3.7393e-02,  2.0345e-02,  ...,  1.9946e-02,
          1.6365e-03,  1.1028e-02],
        [ 6.2553e-03, -3.2888e-02, -4.2043e-03,  ..., -1.5961e-02,
         -3.6810e-02, -1.8867e-02],
        ...,
        [-1.1099e-02, -3.8351e-02,  1.4641e-02,  ...,  2.4120e-03,
         -9.5471e-03, -4.5184e-03],
        [-2.1598e-03,  6.1430e-03,  3.0534e-05,  ...,  2.0336e-02,
          1.0604e-02,  1.4497e-02],
        [ 3.3183e-03,  1.6679e-02, -1.2548e-02,  ..., -1.1251e-02,
         -4.8407e-02,  1.7429e-02]], requires_grad=True)
Pre-trained Encoder Query Weights (Layer 0):
Parameter containing:
tensor([[-0.0496,  0.0298, -0.0069,  ...,  0.0254,  0.0102,  0.0027],
        [ 0.0254,  0.0388,  0.0234,  ...,  0.0176,  0.0007,  0.0110],
        [ 0.0097, -0.0304, -0.0069,  ..., -0.0147, -0.0396, -0.0197],
     

In [109]:
# Calculate statistics for classifier weights
def summarize_weights(weights):
    return {
        "mean": torch.mean(weights).item(),
        "std": torch.std(weights).item(),
        "norm": torch.norm(weights).item()
    }

fine_tuned_classifier_summary = summarize_weights(fine_tuned_classifier_weights)
pretrained_classifier_summary = summarize_weights(pretrained_classifier_weights)

print("Fine-tuned Classifier Weights Summary:")
print(fine_tuned_classifier_summary)

print("Pre-trained Classifier Weights Summary:")
print(pretrained_classifier_summary)

fine_tuned_query_summary = summarize_weights(fine_tuned_query_weights)
pretrained_query_summary = summarize_weights(pretrained_query_weights)

print("Fine-tuned Encoder Query Weights Summary (Layer 0):")
print(fine_tuned_query_summary)

print("Pre-trained Encoder Query Weights Summary (Layer 0):")
print(pretrained_query_summary)

Fine-tuned Classifier Weights Summary:
{'mean': 5.3044414016767405e-06, 'std': 0.01976761408150196, 'norm': 0.9486396312713623}
Pre-trained Classifier Weights Summary:
{'mean': 0.0003834100207313895, 'std': 0.02015182375907898, 'norm': 0.9672526717185974}
Fine-tuned Encoder Query Weights Summary (Layer 0):
{'mean': 4.8687521484680474e-05, 'std': 0.025672325864434242, 'norm': 19.716251373291016}
Pre-trained Encoder Query Weights Summary (Layer 0):
{'mean': 4.6587814722443e-05, 'std': 0.025565769523382187, 'norm': 19.63439178466797}
